## Compare results for multiple results

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

import subprocess as sp
import os
import glob
import sys

import itertools
import time

from ipywidgets import *

In [2]:
%matplotlib widget

In [3]:
sys.path.append('/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/lbann_cosmogan/3_analysis')
from modules_image_analysis import *

In [4]:
### Transformation functions for image pixel values
def f_transform(x):
    return 2.*x/(x + 4. + 1e-8) - 1.

def f_invtransform(s):
    return 4.*(1. + s)/(1. - s + 1e-8)

In [5]:
def f_compute_hist_spect(sample,bins):
    ''' Compute pixel intensity histograms and radial spectrum for 2D arrays
    Input : Image arrays and bins
    Output: dictionary with 5 arrays : Histogram values, errors and bin centers, Spectrum values and errors.
    '''
    ### Compute pixel histogram for row
    gen_hist,gen_err,hist_bins=f_batch_histogram(sample,bins=bins,norm=True,hist_range=None)
    ### Compute spectrum for row
    spec,spec_sdev=f_compute_spectrum(sample,plot=False)

    dict1={'hist_val':gen_hist,'hist_err':gen_err,'hist_bin_centers':hist_bins,'spec_val':spec,'spec_sdev':spec_sdev }
    return dict1

def f_compute_chisqr(dict_val,dict_sample,img_size):
    '''
    Compute chi-square values for sample w.r.t input images
    Input: 2 dictionaries with 4 keys for histogram and spectrum values and errors
    '''
    ### !!Both pixel histograms MUST have same bins and normalization!
    ### Compute chi-sqr
    ### Used in keras code : np.sum(np.divide(np.power(valhist - samphist, 2.0), valhist))
    ###  chi_sqr :: sum((Obs-Val)^2/(Val))
    
    chisqr_dict={}
    
    try: 
        val_dr=dict_val['hist_val'].copy()
        val_dr[val_dr<=0.]=1.0    ### Avoiding division by zero for zero bins

        sq_diff=(dict_val['hist_val']-dict_sample['hist_val'])**2

        size=len(dict_val['hist_val'])
        l1,l2=int(size*0.3),int(size*0.7)
        keys=['chi_1a','chi_1b','chi_1c','chi_1']
        
        for (key,start,end) in zip(keys,[0,l1,l2,0],[l1,l2,None,None]):  # 4 lists : small, medium, large pixel values and full 
            chisqr_dict.update({key:np.sum(np.divide(sq_diff[start:end],val_dr[start:end]))})

        idx=None  # Choosing the number of histograms to use. Eg : -5 to skip last 5 bins
    #     chisqr_dict.update({'chi_sqr1':})

        chisqr_dict.update({'chi_2':np.sum(np.divide(sq_diff[:idx],1.0))}) ## chi-sqr without denominator division
        chisqr_dict.update({'chi_imgvar':np.sum(dict_sample['hist_err'][:idx])/np.sum(dict_val['hist_err'][:idx])}) ## measures total spread in histograms wrt to input data

        idx=int(img_size/2)
        spec_diff=(dict_val['spec_val']-dict_sample['spec_val'])**2
        ### computing the spectral loss chi-square
        chisqr_dict.update({'chi_spec1':np.sum(spec_diff[:idx]/dict_sample['spec_val'][:idx]**2)})

        ### computing the spectral loss chi-square
        chisqr_dict.update({'chi_spec2':np.sum(spec_diff[:idx]/dict_sample['spec_sdev'][:idx]**2)})
        
        spec_loss=1.0*np.log(np.mean((dict_val['spec_val'][:idx]-dict_sample['spec_val'][:idx])**2))+1.0*np.log(np.mean((dict_val['spec_sdev'][:idx]-dict_sample['spec_sdev'][:idx])**2))
        print(spec_loss)
        chisqr_dict.update({'chi_spec3':spec_loss})
    
    except Exception as e: 
        print(e)
        
        keys=['chi_1a','chi_1b','chi_1c','chi_1','chi_2','chi_imgvar','chi_spec1','chi_spec2']
        chisqr_dict=dict.fromkeys(keys,np.nan)
        pass
    
    return chisqr_dict

In [6]:
# def f_compute_hist_spect(sample,bins):
#     ''' Compute pixel intensity histograms and radial spectrum for 2D arrays
#     Input : Image arrays and bins
#     Output: dictionary with 5 arrays : Histogram values, errors and bin centers, Spectrum values and errors.
#     '''
#     ### Compute pixel histogram for row
#     gen_hist,gen_err,hist_bins=f_batch_histogram(sample,bins=bins,norm=True,hist_range=None)
#     ### Compute spectrum for row
#     spec,spec_sdev=f_compute_spectrum(sample,plot=False)

#     dict1={'hist_val':gen_hist,'hist_err':gen_err,'hist_bin_centers':hist_bins,'spec_val':spec,'spec_sdev':spec_sdev }
#     return dict1

# def f_compute_chisqr(dict_val,dict_sample,img_size):
#     '''
#     Compute chi-square values for sample w.r.t input images
#     Input: 2 dictionaries with 4 keys for histogram and spectrum values and errors
#     '''
#     ### !!Both pixel histograms MUST have same bins and normalization!
#     ### Compute chi-sqr
#     ### Used in keras code : np.sum(np.divide(np.power(valhist - samphist, 2.0), valhist))
#     ###  chi_sqr :: sum((Obs-Val)^2/(Val))
    
#     chisqr_dict={}
    
#     try: 
#         val_dr=dict_val['hist_val'].copy()
#         val_dr[val_dr<=0.]=1.0    ### Avoiding division by zero for zero bins

#         sq_diff=(dict_val['hist_val']-dict_sample['hist_val'])**2

#         size=len(dict_val['hist_val'])
#         l1,l2=int(size*0.3),int(size*0.7)
#         keys=['chi_1a','chi_1b','chi_1c','chi_1']
        
#         for (key,start,end) in zip(keys,[0,l1,l2,0],[l1,l2,None,None]):  # 4 lists : small, medium, large pixel values and full 
#             chisqr_dict.update({key:np.sum(np.divide(sq_diff[start:end],val_dr[start:end]))})

#         idx=None  # Choosing the number of histograms to use. Eg : -5 to skip last 5 bins
#     #     chisqr_dict.update({'chi_sqr1':})
    
#         chisqr_dict.update({'chi_2':np.sum(np.divide(sq_diff[:idx],dict_val['hist_err'][:idx]))}) ## chi-sqr 

#         chisqr_dict.update({'chi_imgvar':np.sum(dict_sample['hist_err'][:idx])/np.sum(dict_val['hist_err'][:idx])}) ## measures total spread in histograms wrt to input data

#         idx=int(img_size/2)
#         spec_diff=(dict_val['spec_val']-dict_sample['spec_val'])**2
#         ### computing the spectral loss chi-square
#         chisqr_dict.update({'chi_spec1':np.sum(spec_diff[:idx]/dict_sample['spec_val'][:idx]**2)})

#         ### computing the spectral loss chi-square
#         chisqr_dict.update({'chi_spec2':np.sum(spec_diff[:idx]/dict_sample['spec_sdev'][:idx]**2)})
        
#         spec_loss=1.0*np.log(np.mean((dict_val['spec_val'][:idx]-dict_sample['spec_val'][:idx])**2))+1.0*np.log(np.mean((dict_val['spec_sdev'][:idx]-dict_sample['spec_sdev'][:idx])**2))
#         print(spec_loss)
#         chisqr_dict.update({'chi_spec3':spec_loss})
    
#     except Exception as e: 
#         print(e)
        
#         keys=['chi_1a','chi_1b','chi_1c','chi_1','chi_2','chi_imgvar','chi_spec1','chi_spec2']
#         chisqr_dict=dict.fromkeys(keys,np.nan)
#         pass
    
#     return chisqr_dict

In [7]:
# bins=np.concatenate([np.array([-0.5]),np.arange(0.5,20.5,1),np.arange(20.5,100.5,5),np.arange(100.5,1000.5,50),np.array([2000])]) #bin edges to use
# bins=np.concatenate([np.array([-0.5]),np.arange(0.5,300.5,5),np.arange(300.5,1000.5,50),np.array([2000])]) #bin edges to use
bins=np.concatenate([np.array([-0.5]),np.arange(0.5,100.5,5),np.arange(100.5,300.5,20),np.arange(300.5,1000.5,50),np.array([2000])]) #bin edges to use
bins=f_transform(bins)   ### scale to (-1,1)

### Extract validation data
fname='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_2_smoothing_200k/norm_1_train_val.npy'
# s_val=np.load(fname,mmap_mode='r')[10000:20000][:,0,:,:]
num_bkgnd=8000
s_val=np.load(fname,mmap_mode='r')[:num_bkgnd][:,0,:,:]
print(s_val.shape)
img_size=s_val.shape[1]
### Compute histogram and spectrum of raw data 
dict_val=f_compute_hist_spect(s_val,bins)
del(s_val)

(8000, 128, 128)


/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/lbann_cosmogan/3_analysis/modules_image_analysis.py:85: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  hist_arr=np.array([np.histogram(arr.flatten(), bins=bins, range=(llim,ulim), density=norm) for arr in img_arr]) ## range is important


In [8]:
dict_val.keys()

dict_keys(['hist_val', 'hist_err', 'hist_bin_centers', 'spec_val', 'spec_sdev'])

In [9]:
df_runs=pd.DataFrame([])

In [10]:
### Load raw images ( a different set of input images for comparison)
dict1={}
dict1.update({'label':'raw','fname':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_2_smoothing_200k/norm_1_train_val.npy'})
images=np.load(dict1['fname'],mmap_mode='r')[-500:][:,0,:,:]
dict1['num_imgs']=images.shape[0]
### Compute spectrum and histograms
dict_sample=f_compute_hist_spect(images,bins)
### Compute chi squares
dict_chisqrs=f_compute_chisqr(dict_val,dict_sample,img_size)

dict1.update(dict_sample)
dict1.update(dict_chisqrs)
del(images)
df_runs=df_runs.append(dict1,ignore_index=True)


24.904620728969803


In [11]:
### Load keras images
parent_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/exagan1/'
lst=['run5_fixed_cosmology','run6_fixed_cosmology','run7_no_truncated_normal','run8_no_truncated_normal']
prefix='keras_'

for strg,fldr in zip([str(i) for i in range(len(lst))],lst):
    key=prefix+strg
    fname=parent_dir+fldr+'/models/gen_imgs.npy'
    dict1={'label':key,'fname':fname}

    images=np.load(dict1['fname'])[:500]     ### No transform needed here
#     print(np.max(images),np.min(images))

    dict1['num_imgs']=images.shape[0]
    ### Compute spectrum and histograms
    dict_sample=f_compute_hist_spect(images,bins)
    ### Compute chi squares
    dict_chisqrs=f_compute_chisqr(dict_val,dict_sample,img_size)
    
    dict1.update(dict_sample)
    dict1.update(dict_chisqrs)
    del(images)
    df_runs=df_runs.append(dict1,ignore_index=True)


23.72642724495634
21.61369575112093
25.82132044478256
20.2193980380533


Load LBANN images

In [12]:
parent_dir='/global/cscratch1/sd/vpa/proj/cosmogan/results_dir/128square/20210202_061803_bsize256_scale0.1_deterministic_off'
lst=[(8, 7200), (10, 9570), (12, 10950), (13, 12240), (15, 13390), (15, 13540)]
prefix='lbann_train_1_'
for count,run in enumerate(lst):
    epoch,step=run[0],run[1]
    key=prefix+'{0}-{1}'.format(epoch,step)
    fname=parent_dir+'/dump_outs/trainer0/model0/'+'sgd.training.epoch.{0}.step.{1}_gen_img_instance1_activation_output0.npy'.format(epoch,step)
    dict1={'label':key,'fname':fname}
    images=np.load(fname)[:,0,:,:]
    dict1['num_imgs']=images.shape[0]
    print(dict1)
    ### Compute spectrum and histograms
    dict_sample=f_compute_hist_spect(images,bins)
    ### Compute chi squares
    dict_chisqrs=f_compute_chisqr(dict_val,dict_sample,img_size)
    
    dict1.update(dict_sample)
    dict1.update(dict_chisqrs)
    del(images)
    df_runs=df_runs.append(dict1,ignore_index=True)

{'label': 'lbann_train_1_8-7200', 'fname': '/global/cscratch1/sd/vpa/proj/cosmogan/results_dir/128square/20210202_061803_bsize256_scale0.1_deterministic_off/dump_outs/trainer0/model0/sgd.training.epoch.8.step.7200_gen_img_instance1_activation_output0.npy', 'num_imgs': 256}
29.61504229848409
{'label': 'lbann_train_1_10-9570', 'fname': '/global/cscratch1/sd/vpa/proj/cosmogan/results_dir/128square/20210202_061803_bsize256_scale0.1_deterministic_off/dump_outs/trainer0/model0/sgd.training.epoch.10.step.9570_gen_img_instance1_activation_output0.npy', 'num_imgs': 256}
28.516661118927118
{'label': 'lbann_train_1_12-10950', 'fname': '/global/cscratch1/sd/vpa/proj/cosmogan/results_dir/128square/20210202_061803_bsize256_scale0.1_deterministic_off/dump_outs/trainer0/model0/sgd.training.epoch.12.step.10950_gen_img_instance1_activation_output0.npy', 'num_imgs': 256}
26.84027480223618
{'label': 'lbann_train_1_13-12240', 'fname': '/global/cscratch1/sd/vpa/proj/cosmogan/results_dir/128square/20210202_0

In [13]:
## LBANN generated images

parent_dir='/global/cscratch1/sd/vpa/proj/cosmogan/results_dir/128square/20210202_061803_bsize256_scale0.1_deterministic_off/gen_imgs_chkpt/'
fldr='batch_begin.epoch.8.step.7200_20210203_112317'
epochs=9

prefix='lbann_gen_1_'
for count,epoch in enumerate(range(epochs)):
    key=prefix+'{0}'.format(epoch)
    fname=parent_dir+fldr+'/dump_outs/trainer0/model0/sgd.testing.epoch.{0}.step.{0}_gen_img_instance1_activation_output0.npy'.format(epoch)
    
    dict1={'label':key,'fname':fname}
    images=np.load(fname)[:,0,:,:]
    dict1['num_imgs']=images.shape[0]
    print(dict1)
    ### Compute spectrum and histograms
    dict_sample=f_compute_hist_spect(images,bins)
    ### Compute chi squares
    dict_chisqrs=f_compute_chisqr(dict_val,dict_sample,img_size)
    
    dict1.update(dict_sample)
    dict1.update(dict_chisqrs)
#     del(images)
    df_runs=df_runs.append(dict1,ignore_index=True)
    


{'label': 'lbann_gen_1_0', 'fname': '/global/cscratch1/sd/vpa/proj/cosmogan/results_dir/128square/20210202_061803_bsize256_scale0.1_deterministic_off/gen_imgs_chkpt/batch_begin.epoch.8.step.7200_20210203_112317/dump_outs/trainer0/model0/sgd.testing.epoch.0.step.0_gen_img_instance1_activation_output0.npy', 'num_imgs': 1000}
32.16840785065814
{'label': 'lbann_gen_1_1', 'fname': '/global/cscratch1/sd/vpa/proj/cosmogan/results_dir/128square/20210202_061803_bsize256_scale0.1_deterministic_off/gen_imgs_chkpt/batch_begin.epoch.8.step.7200_20210203_112317/dump_outs/trainer0/model0/sgd.testing.epoch.1.step.1_gen_img_instance1_activation_output0.npy', 'num_imgs': 1000}
32.34362311899937
{'label': 'lbann_gen_1_2', 'fname': '/global/cscratch1/sd/vpa/proj/cosmogan/results_dir/128square/20210202_061803_bsize256_scale0.1_deterministic_off/gen_imgs_chkpt/batch_begin.epoch.8.step.7200_20210203_112317/dump_outs/trainer0/model0/sgd.testing.epoch.2.step.2_gen_img_instance1_activation_output0.npy', 'num_im

In [14]:
## LBANN generated images

parent_dir='/global/cscratch1/sd/vpa/proj/cosmogan/results_dir/128square/20210202_061803_bsize256_scale0.1_deterministic_off/gen_imgs_chkpt/'
fldr='batch_begin.epoch.10.step.9570_20210203_112317'
epochs=9

prefix='lbann_gen_2_'
for count,epoch in enumerate(range(epochs)):
    key=prefix+'{0}'.format(epoch)
    fname=parent_dir+fldr+'/dump_outs/trainer0/model0/sgd.testing.epoch.{0}.step.{0}_gen_img_instance1_activation_output0.npy'.format(epoch)
    
    dict1={'label':key,'fname':fname}
    images=np.load(fname)[:,0,:,:]
    dict1['num_imgs']=images.shape[0]
    print(dict1)
    ### Compute spectrum and histograms
    dict_sample=f_compute_hist_spect(images,bins)
    ### Compute chi squares
    dict_chisqrs=f_compute_chisqr(dict_val,dict_sample,img_size)
    
    dict1.update(dict_sample)
    dict1.update(dict_chisqrs)
#     del(images)
    df_runs=df_runs.append(dict1,ignore_index=True)
    

{'label': 'lbann_gen_2_0', 'fname': '/global/cscratch1/sd/vpa/proj/cosmogan/results_dir/128square/20210202_061803_bsize256_scale0.1_deterministic_off/gen_imgs_chkpt/batch_begin.epoch.10.step.9570_20210203_112317/dump_outs/trainer0/model0/sgd.testing.epoch.0.step.0_gen_img_instance1_activation_output0.npy', 'num_imgs': 1000}
29.324139161977428
{'label': 'lbann_gen_2_1', 'fname': '/global/cscratch1/sd/vpa/proj/cosmogan/results_dir/128square/20210202_061803_bsize256_scale0.1_deterministic_off/gen_imgs_chkpt/batch_begin.epoch.10.step.9570_20210203_112317/dump_outs/trainer0/model0/sgd.testing.epoch.1.step.1_gen_img_instance1_activation_output0.npy', 'num_imgs': 1000}
28.563177574662568
{'label': 'lbann_gen_2_2', 'fname': '/global/cscratch1/sd/vpa/proj/cosmogan/results_dir/128square/20210202_061803_bsize256_scale0.1_deterministic_off/gen_imgs_chkpt/batch_begin.epoch.10.step.9570_20210203_112317/dump_outs/trainer0/model0/sgd.testing.epoch.2.step.2_gen_img_instance1_activation_output0.npy', 'n

In [15]:
## LBANN generated images

parent_dir='/global/cscratch1/sd/vpa/proj/cosmogan/results_dir/128square/20210202_061803_bsize256_scale0.1_deterministic_off/gen_imgs_chkpt/'
fldr='batch_begin.epoch.13.step.12240_20210203_071523'
epochs=9

prefix='lbann_gen_3_'
for count,epoch in enumerate(range(epochs)):
    key=prefix+'{0}'.format(epoch)
    fname=parent_dir+fldr+'/dump_outs/trainer0/model0/sgd.testing.epoch.{0}.step.{0}_gen_img_instance1_activation_output0.npy'.format(epoch)
    
    dict1={'label':key,'fname':fname}
    images=np.load(fname)[:,0,:,:]
    dict1['num_imgs']=images.shape[0]
    print(dict1)
    ### Compute spectrum and histograms
    dict_sample=f_compute_hist_spect(images,bins)
    ### Compute chi squares
    dict_chisqrs=f_compute_chisqr(dict_val,dict_sample,img_size)
    
    dict1.update(dict_sample)
    dict1.update(dict_chisqrs)
#     del(images)
    df_runs=df_runs.append(dict1,ignore_index=True)
    

{'label': 'lbann_gen_3_0', 'fname': '/global/cscratch1/sd/vpa/proj/cosmogan/results_dir/128square/20210202_061803_bsize256_scale0.1_deterministic_off/gen_imgs_chkpt/batch_begin.epoch.13.step.12240_20210203_071523/dump_outs/trainer0/model0/sgd.testing.epoch.0.step.0_gen_img_instance1_activation_output0.npy', 'num_imgs': 1000}
24.825189340368652
{'label': 'lbann_gen_3_1', 'fname': '/global/cscratch1/sd/vpa/proj/cosmogan/results_dir/128square/20210202_061803_bsize256_scale0.1_deterministic_off/gen_imgs_chkpt/batch_begin.epoch.13.step.12240_20210203_071523/dump_outs/trainer0/model0/sgd.testing.epoch.1.step.1_gen_img_instance1_activation_output0.npy', 'num_imgs': 1000}
24.822727022113913
{'label': 'lbann_gen_3_2', 'fname': '/global/cscratch1/sd/vpa/proj/cosmogan/results_dir/128square/20210202_061803_bsize256_scale0.1_deterministic_off/gen_imgs_chkpt/batch_begin.epoch.13.step.12240_20210203_071523/dump_outs/trainer0/model0/sgd.testing.epoch.2.step.2_gen_img_instance1_activation_output0.npy',

In [19]:
col_list=['label','fname', 'hist_bin_centers', 'hist_err', 'hist_val','spec_sdev', 'spec_val','chi_1', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2',
       'chi_imgvar', 'chi_spec1', 'chi_spec2','chi_spec3', 'num_imgs']
df_runs=df_runs.reindex(columns=col_list)


In [20]:
# df_runs

### Compare chi-squares

In [21]:
chi_keys=['label','chi_1','chi_spec1', 'chi_spec3', 'num_imgs', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2','chi_imgvar', 'chi_spec2','fname']
df_runs[chi_keys]

,label,chi_1,chi_spec1,chi_spec3,num_imgs,chi_1a,chi_1b,chi_1c,chi_2,chi_imgvar,chi_spec2,fname
0,raw,0.002139,0.005615,24.904621,500.0,0.000124,0.001016,0.000998,0.000040,3.823114,0.217226,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...
1,keras_0,0.001611,0.054668,23.726427,500.0,0.000093,0.000604,0.000914,0.000020,4.096589,5.546397,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...
2,keras_1,0.001482,0.070094,21.613696,500.0,0.000096,0.000321,0.001064,0.000019,4.130560,6.117566,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...
3,keras_2,0.002356,0.020309,25.821320,500.0,0.001164,0.000546,0.000646,0.000868,4.045721,1.257560,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...
4,keras_3,0.002777,0.050543,20.219398,500.0,0.000815,0.000796,0.001166,0.000563,4.002570,5.391138,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...
5,lbann_train_1_8-7200,0.076392,0.543510,29.615042,256.0,0.002419,0.000562,0.073411,0.000615,7.276323,58.662930,/global/cscratch1/sd/vpa/proj/cosmogan/results...
6,lbann_train_1_10-9570,0.044202,0.348262,28.516661,256.0,0.001671,0.000481,0.042050,0.000103,6.278294,31.512938,/global/cscratch1/sd/vpa/proj/cosmogan/results...
7,lbann_train_1_12-10950,0.011643,0.489623,26.840275,256.0,0.002727,0.006557,0.002358,0.000723,5.430217,41.068094,/global/cscratch1/sd/vpa/proj/cosmogan/results...
8,lbann_train_1_13-12240,0.010629,0.574117,25.791356,256.0,0.001359,0.008560,0.000710,0.000116,5.931152,51.603151,/global/cscratch1/sd/vpa/proj/cosmogan/results...
9,lbann_train_1_15-13390,0.044513,0.280854,26.753791,256.0,0.019133,0.002511,0.022869,0.013029,6.482032,29.028585,/global/cscratch1/sd/vpa/proj/cosmogan/results...


In [22]:
dict_val['spec_val']
dict_val.keys()


dict_keys(['hist_val', 'hist_err', 'hist_bin_centers', 'spec_val', 'spec_sdev'])

In [23]:

def f_plot_hist_spec(df,dict_bkg,plot_type):

    img_size=128
    assert plot_type in ['hist','spec','spec_relative','grid'],"Invalid mode %s"%(plot_type)

    if plot_type in ['hist','spec','spec_relative']:     fig=plt.figure(figsize=(6,6))
    
    for (i,row),marker in zip(df.iterrows(),itertools.cycle('>^*sDHPdpx_')):
        label=row.label
        if plot_type=='hist':
            x1=row.hist_bin_centers
            y1=row.hist_val
            yerr1=row.hist_err
            x1=f_invtransform(x1)
            
            plt.errorbar(x1,y1,yerr1,marker=marker,markersize=5,linestyle='',label=label)
        if plot_type=='spec':
            
            y2=row.spec_val
            yerr2=row.spec_sdev/np.sqrt(row.num_imgs)
            x2=np.arange(len(y2))
            
            plt.fill_between(x2, y2 - yerr2, y2 + yerr2, alpha=0.4)
            plt.plot(x2, y2, marker=marker, linestyle=':',label=label)

        if plot_type=='spec_relative':
            
            y2=row.spec_val
            yerr2=row.spec_sdev
            x2=np.arange(len(y2))
            
            ### Reference spectrum
            y1,yerr1=dict_bkg['spec_val'],dict_bkg['spec_sdev']
            y=y2/(1.0*y1)
            ## Variance is sum of variance of both variables, since they are uncorrelated
            
            # delta_r= |r| * sqrt(delta_a/a)^2 +(\delta_b/b)^2) / \sqrt(N)
            yerr=(np.abs(y))*np.sqrt((yerr1/y1)**2+(yerr2/y2)**2)/np.sqrt(row.num_imgs)

            plt.fill_between(x2, y - yerr, y + yerr, alpha=0.4)
            plt.plot(x2, y, marker=marker, linestyle=':',label=label)
            plt.xlim(0,img_size/2)
            plt.ylim(0.5,2)
            plt.legend()
        
        if plot_type=='grid':
            images=np.load(row.fname)[:,0,:,:]
            f_plot_grid(images[:18],cols=6,fig_size=(10,5))
            
    ### Plot input data
    if plot_type=='hist':
        x,y,yerr=dict_bkg['hist_bin_centers'],dict_bkg['hist_val'],dict_bkg['hist_err']
        x=f_invtransform(x)
        plt.errorbar(x, y,yerr,color='k',linestyle='-',label='bkgnd')   
        plt.title('Pixel Intensity Histogram')
        plt.xscale('symlog',linthreshx=300)
        
    if plot_type=='spec':
        y,yerr=dict_bkg['spec_val'],dict_bkg['spec_sdev']/np.sqrt(num_bkgnd)
        x=np.arange(len(y))
        plt.fill_between(x, y - yerr, y + yerr, color='k',alpha=0.8)
        plt.title('Spectrum')
        
    if plot_type=='spec_relative':
        plt.axhline(y=1.0,color='k',linestyle='-.')
    
    plt.legend(bbox_to_anchor=(0.3, 0.75),ncol=2, fancybox=True, shadow=True,prop={'size':6})
    if plot_type in ['hist','spec']:     plt.yscale('log')    
    

# f_plot_hist_spec(df_runs,dict_val,'hist')
# f_plot_hist_spec(df_runs,dict_val,'spec')
# f_plot_hist_spec(df_best,dict_val,'grid')



In [24]:
def f_widget_compare(df,labels_list,bkgnd,plot_type):
    df_temp=df[df.label.isin(labels_list)]
    f_plot_hist_spec(df_temp,dict_val,plot_type)
    
interact_manual(f_widget_compare,df=fixed(df_runs),
                labels_list=SelectMultiple(options=df_runs.label.values),bkgnd=fixed(dict_val),
                plot_type=ToggleButtons(options=['hist','spec','grid','spec_relative']))

interactive(children=(SelectMultiple(description='labels_list', options=('raw', 'keras_0', 'keras_1', 'keras_2…

<function __main__.f_widget_compare(df, labels_list, bkgnd, plot_type)>

In [21]:
col_list=['label','chi_1','chi_spec1','num_imgs','chi_2','chi_spec2']
df_runs[col_list]

,label,chi_1,chi_spec1,num_imgs,chi_2,chi_spec2
0,raw,0.002139,0.005615,500.0,0.000040,0.217226
1,keras_0,0.001611,0.054668,500.0,0.000020,5.546397
2,keras_1,0.001482,0.070094,500.0,0.000019,6.117566
3,keras_2,0.002356,0.020309,500.0,0.000868,1.257560
4,keras_3,0.002777,0.050543,500.0,0.000563,5.391138
5,lbann_train_1_8-7200,0.076392,0.543510,256.0,0.000615,58.662930
6,lbann_train_1_10-9570,0.044202,0.348262,256.0,0.000103,31.512938
7,lbann_train_1_12-10950,0.011643,0.489623,256.0,0.000723,41.068094
8,lbann_train_1_13-12240,0.010629,0.574117,256.0,0.000116,51.603151
9,lbann_train_1_15-13390,0.044513,0.280854,256.0,0.013029,29.028585


In [22]:
# f_plot_intensity_grid(images[:20],cols=5)
# f_pixel_intensity(f_transform(images),bins=100)



In [ ]:
fname='/global/cscratch1/sd/vpa/proj/cosmogan/results_dir/128square/20210202_061803_bsize256_scale0.1_deterministic_off/gen_imgs_chkpt/batch_begin.epoch.13.step.12240_20210203_071523/dump_outs/trainer0/model0/sgd.testing.epoch.0.step.0_gen_img_instance1_activation_output0.npy'
a1=np.load(fname)

In [ ]:
a1.shape

In [ ]:
np.max(a1),np.min(a1)

In [ ]:
np.histogram(a1)